# 파이썬 기반의 AI를 위한 기초수학, 확률및통계

In [3]:
## 강봉주 
## bonjour.kang@gmail.com
## 조건부 확률과 독립
##

In [4]:
# 필요한 패키지
import numpy as np
import scipy
from scipy import linalg as la
import matplotlib.pyplot as plt

import scipy.stats as ss
import sympy

# 한글출력
plt.rcParams['font.family'] = 'AppleGothic' #  Windows 'Malgun Gothic' 
plt.rcParams['axes.unicode_minus'] = False

# 필요한 패키지 2
from pgmpy.factors.discrete import JointProbabilityDistribution as JPD
import pgmpy
pgmpy.__version__

'1.0.0'

# 조건부 확률의 개념
- 어떤 사건 a가 이미 일어났다는 조건 하에 사건 b가 일어날 확률
- 예시
  + 전체 학생 중 60%가 수학을 좋아함, P(A) = 0.6 (수학을 좋아하지 앟ㄴ는 확률 0.4)
  + 수학을 좋아하는 학생의 70%가 시험에 합격 P(B|A) = 0.7
  + 질문: 전체 학생 중, 수학을 좋아하면서 시험에 합격한 비율은?

In [5]:
P_A = 0.6
P_B_given_A = 0.7

# 조건부 확률 공식
P_A_and_B = P_A * P_B_given_A
print(P_A_and_B)

0.42


## 주변 분포와 조건부 분포
- 다변량 데이터 분석을 활용할 때도 조건부 확률의 개념이 들어가지 않을까? 추측
  + 상관분석, 회귀분석

- p.155
- p.158

In [6]:
# 결합 확률 정의
from pgmpy.factors.discrete import JointProbabilityDistribution as JPD

## 하나의 사건만을 가지고 확률 얘기
# 지금은 두 개의 사건을 조합 해서 확률 값을 구할 수 있다.
# 결합확률밀도 함수
prob = list()
for i in np.arange(1, 4):
    for j in np.arange(1,3):
        prob.append((i+j)/21)
        
fxy = JPD(['X1', 'X2'],[3, 2], prob)
print(fxy)

+-------+-------+------------+
| X1    | X2    |   P(X1,X2) |
+=======+=======+============+
| X1(0) | X2(0) |     0.0952 |
+-------+-------+------------+
| X1(0) | X2(1) |     0.1429 |
+-------+-------+------------+
| X1(1) | X2(0) |     0.1429 |
+-------+-------+------------+
| X1(1) | X2(1) |     0.1905 |
+-------+-------+------------+
| X1(2) | X2(0) |     0.1905 |
+-------+-------+------------+
| X1(2) | X2(1) |     0.2381 |
+-------+-------+------------+


In [7]:
# 확률 밀도 함수 확인
np.sum(fxy.values)

np.float64(1.0)

In [8]:
# 주변 확률 구하기
fx = fxy.marginal_distribution(['X1'], inplace=False)
print(fx)

+-------+---------+
| X1    |   P(X1) |
+=======+=========+
| X1(0) |  0.2381 |
+-------+---------+
| X1(1) |  0.3333 |
+-------+---------+
| X1(2) |  0.4286 |
+-------+---------+


[예제] $X_1=1$ 일 때 $X_2$ 의 조건부 확률

In [11]:
prob = fxy.conditional_distribution([('X1', 0)], inplace=False)
print(prob)

ValueError: The variable: X1 is not in the factor

## 상관계수

In [12]:
# 결합확률 정의
prob = np.array([2, 4, 3, 1, 1, 4])/15
fxy = JPD(['X', 'Y'],[2, 3], prob)
print(fxy)

+------+------+----------+
| X    | Y    |   P(X,Y) |
+======+======+==========+
| X(0) | Y(0) |   0.1333 |
+------+------+----------+
| X(0) | Y(1) |   0.2667 |
+------+------+----------+
| X(0) | Y(2) |   0.2000 |
+------+------+----------+
| X(1) | Y(0) |   0.0667 |
+------+------+----------+
| X(1) | Y(1) |   0.0667 |
+------+------+----------+
| X(1) | Y(2) |   0.2667 |
+------+------+----------+


## 결합 확률밀도함수 시각화, 해석

In [13]:
x = np.linspace(0,1,100) # 독립적인 사건, 서로 다른 수치형 필드
y = np.linspace(0,1,100) # 독립적인 사건, 서로 다른 수치형 필드
X, Y = np.meshgrid(x,y)

# 결합 확률 밀도 함수
Z = np.ones_like(X) # 모든 구간에서 밀도는 1 # 특정한 시공간이 있는데, 데이터의 밀도는 일정
Z


array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], shape=(100, 100))

In [8]:
# 확률 분포 확인
np.sum(fxy.values)

np.float64(1.0)

예시 강아지, 고양이, 새
- 특정한 이미지를 투영하면
  + 사람 : 이 이미지는 강아지, 고양이, 새
  + 딥러닝 : 강아지일 확률 60%, 고양이일 확률 30%, 새일 확률 10%
    - 결과는 강아지다.

In [9]:
# X 주변 확률 구하기
# 딥러닝 softmax 함수 확률
# 다중분류 나오는 개념 유시
fx = fxy.marginal_distribution(['X'], inplace=False)
print(fx)

+------+--------+
| X    |   P(X) |
+======+========+
| X(0) | 0.6000 |
+------+--------+
| X(1) | 0.4000 |
+------+--------+


In [10]:
# X의 기대값 구하기
x = np.array([1, 2])
EX = np.dot(x,fx.values )
EX.round(3)

np.float64(1.4)

In [11]:
# X의 제곱 기대값 구하기
EXX = np.dot(x*x, fx.values)
EXX.round(3)

np.float64(2.2)

In [12]:
# X의 표준편차 구하기
SDX = np.sqrt(EXX-EX**2)
SDX.round(3)

np.float64(0.49)

In [13]:
# Y의 주변 분포, 기대값, 제곱합, 표준편차

# Y의 주변 분포
fy = fxy.marginal_distribution(['Y'], inplace=False)

# Y의 기대값
y = np.array([1, 2, 3])
EY = np.dot(y,fy.values )

# Y의 제곱 기대값
EYY = np.dot(y*y, fy.values)

# Y의 표준편차
SDY = np.sqrt(EYY-EY**2)

In [14]:
# 공분산 구하기
# xy 값 구성
xy = np.outer(x, y).reshape(-1,)
xy

array([1, 2, 3, 2, 4, 6])

In [15]:
# XY 기대값
EXY = np.dot(xy, fxy.values.reshape(-1, ))
EXY.round(3)

np.float64(3.267)

In [16]:
# 공분산 계산
CovXY = EXY - EX*EY
CovXY.round(3)

np.float64(0.093)

In [17]:
# 상관계수 값
rho = CovXY / (SDX * SDY)
rho.round(3)

np.float64(0.247)

In [18]:
# 두 변수 간의 독립성 여부 확인
fxy.check_independence(['X'], ['Y'])

False